<a href="https://colab.research.google.com/github/WilliamShengYangHuang/Making-Design-with-Pix2Pix/blob/main/Pix2Pix_feature_map_ebird_h_w_rep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Making Design with Pix2Pix** 

Original Development: Image-to-Image Translation with Conditional Adversarial Nets: https://phillipi.github.io/pix2pix/

Paper available at https://arxiv.org/abs/1611.07004


Modified from the version published at https://www.tensorflow.org/tutorials/generative/pix2pix
(Last modified on 09 July 2022 by William Huang (ucfnhua@ucl.ac.uk) and Chaleston Song)

## Import libraries

In [ ]:
!pip install tensorflow==2.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import sys
import time
import numpy as np

import tensorflow as tf
from matplotlib import pyplot as plt
%matplotlib inline
from IPython import display

from google.colab import drive

In [ ]:
print(tf.__version__)

2.1.0


## List devices

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 7966188195518369040, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 13334990298518849823
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 4264381963628412916
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15791672525
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 17892289304206460466
 physical_device_desc: "device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:04.0, compute capability: 7.0"]

## Load the dataset

In [ ]:
# Place the dataset in your Google Drive 
drive.mount("/content/drive")

######## set dataset path ########-------------------------------------------------------------------------
# folder_name = 'old_hwang/dataset' #@param{type:"string"}
base_dir = "/content/drive/My Drive/"
# path = "/content/drive/My Drive/" + folder_name 
# path = "/content/drive/My Drive/ml_test/0.jpg

folder_name = 'phd_thesis_data/deep_city/pix2pix/compiled/ebird/ebird_h_w' #@param{type:"string"}
path = os.path.join(base_dir, folder_name) 
checkpoint_dir = os.path.join(os.getcwd(),'training_checkpoints')
# os.chdir(path) # change the current working directory to the given path
os.listdir(path) # list all the entries in the directory given by path

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: ignored

In [ ]:
BUFFER_SIZE = 400
BATCH_SIZE = 1
IMG_WIDTH = 256
IMG_HEIGHT = 256
print(path)

def load(image_file):
  image = tf.io.read_file(image_file)
  image = tf.image.decode_jpeg(image)

  w = tf.shape(image)[1]

  w = w // 2
  real_image = image[:, w:, :] # right image
  input_image = image[:, :w, :] # left image

  input_image = tf.cast(input_image, tf.float32)
  real_image = tf.cast(real_image, tf.float32)

  return input_image, real_image

## Plot input-target sample 

In [ ]:
file_name = 16 #@param {type:"slider", min:0, max:29, step:1} 
file_format = '.jpg' #@param [".jpg", ".png", ".bmp"] {allow-input: true}
inp, re = load(path + '/train/' + str(file_name) + file_format) ######## paste the path to one any image within the training set ########---------------------------------------------------------------

# casting to int for matplotlib to show the image
plt.figure()
#plt.axis('off')
plt.imshow(inp/255.0)

plt.figure()
#plt.axis('off')
plt.imshow(re/255.0)

## Data preprocessing & augmentation

In [ ]:
def resize(input_image, real_image, height, width):
  input_image = tf.image.resize(input_image, [height, width],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  real_image = tf.image.resize(real_image, [height, width],
                               method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

  return input_image, real_image

def random_crop(input_image, real_image):
  stacked_image = tf.stack([input_image, real_image], axis=0)
  cropped_image = tf.image.random_crop(
      stacked_image, size=[2, IMG_HEIGHT, IMG_WIDTH, 3])

  return cropped_image[0], cropped_image[1]

# normalising the images to [-1, 1]
def normalize(input_image, real_image):
  input_image = (input_image / 127.5) - 1
  real_image = (real_image / 127.5) - 1

  return input_image, real_image

@tf.function()
def random_jitter(input_image, real_image):
  # resising to 286 x 286 x 3
  input_image, real_image = resize(input_image, real_image, 286, 286)

  # randomly cropping to 256 x 256 x 3
  input_image, real_image = random_crop(input_image, real_image)

  if tf.random.uniform(()) > 0.5:
    # random mirroring
    input_image = tf.image.flip_left_right(input_image)
    real_image = tf.image.flip_left_right(real_image)

  return input_image, real_image

In [ ]:
# inspect some of the preprocessed output
plt.figure(figsize=(6, 6))
for i in range(4):
  rj_inp, rj_re = random_jitter(inp, re)
  plt.subplot(2, 2, i+1)
  plt.imshow(rj_inp/255.0)
  plt.axis('off')
plt.show()

In [ ]:
def load_image_train(image_file):
  input_image, real_image = load(image_file)
  input_image, real_image = random_jitter(input_image, real_image)
  input_image, real_image = normalize(input_image, real_image)

  return input_image, real_image

def load_image_test(image_file):
  input_image, real_image = load(image_file)
  input_image, real_image = resize(input_image, real_image,
                                   IMG_HEIGHT, IMG_WIDTH)
  input_image, real_image = normalize(input_image, real_image)

  return input_image, real_image

def load_image_val(image_file):
  input_image, real_image = load(image_file)
  input_image, real_image = resize(input_image, real_image,
                                   IMG_HEIGHT, IMG_WIDTH)
  input_image, real_image = normalize(input_image, real_image)

  return input_image, real_image

## Input pipeline

In [ ]:
train_dataset = tf.data.Dataset.list_files(path+'/train/*.jpg')
# train_dataset = train_dataset.map(load_image_train, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE)

In [ ]:
test_dataset = tf.data.Dataset.list_files(path+'/test/*.jpg')
test_dataset = test_dataset.map(load_image_test)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
val_dataset = tf.data.Dataset.list_files(path+'/val/*.jpg')
val_dataset = val_dataset.map(load_image_val)
val_dataset = val_dataset.batch(BATCH_SIZE)

## Build generator (U-Net)

In [ ]:
OUTPUT_CHANNELS = 3

def downsample(filters, size, apply_batchnorm=True):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))

  if apply_batchnorm:
    result.add(tf.keras.layers.BatchNormalization())

  result.add(tf.keras.layers.LeakyReLU())

  return result

down_model = downsample(3, 4)
down_result = down_model(tf.expand_dims(inp, 0))
print (down_result.shape)

def upsample(filters, size, apply_dropout=False):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))

  result.add(tf.keras.layers.BatchNormalization())

  if apply_dropout:
      result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU())

  return result

up_model = upsample(3, 4)
up_result = up_model(down_result)
print (up_result.shape)

In [ ]:
def Generator():
  inputs = tf.keras.layers.Input(shape=[256,256,3])

  down_stack = [
    downsample(64, 4, apply_batchnorm=False), # (bs, 128, 128, 64)
    downsample(128, 4), # (bs, 64, 64, 128)
    downsample(256, 4), # (bs, 32, 32, 256)
    downsample(512, 4), # (bs, 16, 16, 512)
    downsample(512, 4), # (bs, 8, 8, 512)
    downsample(512, 4), # (bs, 4, 4, 512)
    downsample(512, 4), # (bs, 2, 2, 512)
    downsample(512, 4), # (bs, 1, 1, 512)
  ]

  up_stack = [
    upsample(512, 4, apply_dropout=True), # (bs, 2, 2, 1024)
    upsample(512, 4, apply_dropout=True), # (bs, 4, 4, 1024)
    upsample(512, 4, apply_dropout=True), # (bs, 8, 8, 1024)
    upsample(512, 4), # (bs, 16, 16, 1024)
    upsample(256, 4), # (bs, 32, 32, 512)
    upsample(128, 4), # (bs, 64, 64, 256)
    upsample(64, 4), # (bs, 128, 128, 128)
  ]

  initializer = tf.random_normal_initializer(0., 0.02)
  last = tf.keras.layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='tanh') # (bs, 256, 256, 3)

  x = inputs

  # Downsampling through the model
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = tf.keras.layers.Concatenate()([x, skip])

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
generator = Generator()
generator.summary()
tf.keras.utils.plot_model(generator, show_shapes=True, dpi=64)

In [ ]:
gen_output = generator(inp[tf.newaxis,...], training=False)
plt.imshow(gen_output[0,...])

## Build discriminator

In [ ]:
LAMBDA = 100
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(disc_generated_output, gen_output, target):
  gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

  # mean absolute error
  l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

  total_gen_loss = gan_loss + (LAMBDA * l1_loss)

  return total_gen_loss, gan_loss, l1_loss

def Discriminator():
  initializer = tf.random_normal_initializer(0., 0.02)

  inp = tf.keras.layers.Input(shape=[256, 256, 3], name='input_image')
  tar = tf.keras.layers.Input(shape=[256, 256, 3], name='target_image')

  x = tf.keras.layers.concatenate([inp, tar]) # (bs, 256, 256, channels*2)

  down1 = downsample(64, 4, False)(x) # (bs, 128, 128, 64)
  down2 = downsample(128, 4)(down1) # (bs, 64, 64, 128)
  down3 = downsample(256, 4)(down2) # (bs, 32, 32, 256)

  zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3) # (bs, 34, 34, 256)
  conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                kernel_initializer=initializer,
                                use_bias=False)(zero_pad1) # (bs, 31, 31, 512)

  batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

  leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

  zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu) # (bs, 33, 33, 512)

  last = tf.keras.layers.Conv2D(1, 4, strides=1, kernel_initializer=initializer)(zero_pad2) # (bs, 30, 30, 1)

  return tf.keras.Model(inputs=[inp, tar], outputs=last)

discriminator = Discriminator()
tf.keras.utils.plot_model(discriminator, show_shapes=True, dpi=64)

disc_out = discriminator([inp[tf.newaxis,...], gen_output], training=False)
plt.imshow(disc_out[0,...,-1], vmin=-20, vmax=20, cmap='RdBu_r')
plt.colorbar()



## Define loss functions and optimiser

In [ ]:
def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [ ]:
# set checkpoints
checkpoint_dir = path + '/training_checkpoints/'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## Generate images

In [ ]:
def generate_images(model, test_input, tar):
  prediction = model(test_input, training=True)
  plt.figure(figsize=(15,15))

  display_list = [test_input[0], tar[0], prediction[0]]

  ######## Edit image titles (modify as needed)########------------------------
  title_domain_A = 'Connectivity (Humans)' #@param {type:"string"}
  title_domain_B = 'Connectivity (Wildlife)' #@param {type:"string"}
  title_prediction = 'Predicted Connectivity (Wildlife)' #@param {type:"string"}
  title = [title_domain_A, title_domain_B, title_prediction] 

  for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.title(title[i])
    # getting the pixel values between [0, 1] to plot it.
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.savefig('generated_image.png')  
  plt.show()  

for example_input, example_target in test_dataset.take(1):
  generate_images(generator, example_input, example_target)  

## Training

In [ ]:
EPOCHS = 1000 #@param {type:"slider", min:1, max:1000, step:1} 
######## set epochs number ########--------------------------------------------

import datetime
log_dir=path + "/logs/"

summary_writer = tf.summary.create_file_writer(
  log_dir + "fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [ ]:
### @tf.function
def train_step(input_image, target, epoch):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generator(input_image, training=True)

    disc_real_output = discriminator([input_image, target], training=True)
    disc_generated_output = discriminator([input_image, gen_output], training=True)

    gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  generator_gradients = gen_tape.gradient(gen_total_loss, generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients, generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients, discriminator.trainable_variables))

  #   with summary_writer.as_default():
  #     tf.summary.scalar('gen_total_loss', gen_total_loss, step=epoch)
  #     tf.summary.scalar('gen_gan_loss', gen_gan_loss, step=epoch)
  #     tf.summary.scalar('gen_l1_loss', gen_l1_loss, step=epoch)
  #     tf.summary.scalar('disc_loss', disc_loss, step=epoch)

  return gen_total_loss.numpy(), gen_gan_loss.numpy(), gen_l1_loss.numpy(), disc_loss.numpy()

In [ ]:
def fit(train_ds, epochs, test_ds):
    gen_total_losses, gen_gan_losses, gen_l1_losses, disc_losses = [], [], [], []
    for epoch in range(epochs):
      gen_total_loss, gen_gan_loss, gen_l1_loss, disc_loss = 0, 0, 0, 0

      start = time.time()

      #         display.clear_output(wait=True)

      #for example_input, example_target in test_ds.take(1):
        #generate_images(generator, example_input, example_target) #--------------------

      #         Train
      i = 1
      for n, (input_image, target) in train_ds.enumerate():

        print('.', end='')
        if (n + 1) % 100 == 0:
          print()
        gen_total_loss_n, gen_gan_loss_n, gen_l1_loss_n, disc_loss_n = train_step(input_image, target, epoch)
        gen_total_loss += gen_total_loss_n
        gen_gan_loss += gen_gan_loss_n
        gen_l1_loss += gen_l1_loss_n
        disc_loss += disc_loss_n
        i += 1

      gen_total_loss /= i
      gen_gan_loss /= i
      gen_l1_loss /= i
      disc_loss /= i
      gen_total_losses.append(gen_total_loss)
      gen_gan_losses.append(gen_gan_loss)
      gen_l1_losses.append(gen_l1_loss)
      disc_losses.append(disc_loss)

      print('epoch {0}: gen_total_loss={1:4f}, gen_gan_loss={2:4f}, gen_l1_loss={3:4f}, disc_loss={4:4f}'.format(
        epoch, gen_total_loss, gen_gan_loss, gen_l1_loss, disc_loss))

    # saving (checkpoint) the model every 20 epochs ------------------------------------------------------------------------------------
    saving_frequency = 100  # @param {type:"slider", min:1, max:100, step:1}
    if (epoch + 1) % saving_frequency == 0:
      checkpoint.save(file_prefix=checkpoint_prefix)

    print('Time taken for epoch {} is {} sec\n'.format(epoch + 1, time.time() - start))
    return gen_total_losses, gen_gan_losses, gen_l1_losses, disc_losses

In [ ]:
# !pip install -U tensorboard

In [ ]:
# display.IFrame(
#     src="https://tensorboard.dev/experiment/lZ0C6FONROaUMfjYkVyJqw",
#     width="100%",
#     height="1000px")

In [ ]:
#docs_infra: no_execute
# %load_ext tensorboard
# %tensorboard --logdir {log_dir}

gen_total_losses, gen_gan_losses, gen_l1_losses, disc_losses = fit(train_dataset, EPOCHS, test_dataset)

In [ ]:
# !tensorboard dev upload --logdir  {log_dir}

Plot losses

In [ ]:
import plotly
import plotly.express as px
import pandas as pd
def draw_loss_interact(loss_lst, loss_type):
    df = pd.DataFrame(np.array([range(1, EPOCHS+1), loss_lst]).T, columns = ["x", "y"])  # choose column

    fig = px.line(df, x="x", y="y", labels={'x':'Epochs', 'y':'Loss'}, title=loss_type)
    fig.show()

In [ ]:
draw_loss_interact(loss_lst=gen_total_losses, loss_type='Generator total loss')
draw_loss_interact(loss_lst=gen_gan_losses, loss_type='Generator GAN loss')
draw_loss_interact(loss_lst=gen_l1_losses, loss_type='Generator L1 loss')
draw_loss_interact(loss_lst=disc_losses, loss_type='Discriminator loss')

## Restore the checkpoint and draw feature map

In [ ]:
from tensorflow.keras.models import Model

# list all saved checkpoint models
checkpoint_models_name = [f for f in os.listdir(checkpoint_dir) if os.path.isfile(os.path.join(checkpoint_dir, f)) & ('.index' in f)]
print('checkpoint models name:', checkpoint_models_name)

# restore particular checkpoint model
checkpoint.restore(os.path.join(checkpoint_dir,checkpoint_models_name[0]))

In [ ]:
def generate_feature_map(model_type, num_layer, test_input, tar, num_fig_grid_col=16):
    
    if model_type == 'generator':
        # create a new model based on particular layer
        layer_model = Model(inputs=checkpoint.generator.inputs, outputs=checkpoint.generator.layers[num_layer].output)
        
        # prediction via layer_model and checkpoit's weights
        prediction = layer_model(test_input)
        
    elif model_type == 'discriminator':
        # create a new model based on particular layer
        layer_model = Model(inputs=checkpoint.discriminator.inputs, outputs=checkpoint.discriminator.layers[num_layer].output)
        
        # prediction via layer_model and checkpoit's weights
        prediction = layer_model([test_input, tar])
        
    else:
        raise ValueError('type error, should be generator or discriminator')    
    
    # check size of prediction
    pred_size = prediction.shape[-1]
    
    # normalize prediction
    prediction = (prediction -np.nanmin(prediction)) / (np.nanmax(prediction) - np.nanmin(prediction)) *2 -1

    # print(prediction)
    num_fig_grid_row = int(np.ceil(pred_size/num_fig_grid_col))  

    # fig.set_size_inches(30,30)    
    print('{0} with layer {1}:'.format(model_type,num_layer))

    # draw input image
    plt.figure(figsize=(15/num_fig_grid_col, 15/num_fig_grid_col))
    plt.axis('off')
    plt.imshow(test_input[0]*0.5+0.5)
    plt.title('Input image', fontsize=8)

    fig, ax = plt.subplots(num_fig_grid_row, num_fig_grid_col)  # figure grid size = (num_fig_grid_row, num_fig_grid_col)
    fig.set_size_inches(30, 30/num_fig_grid_col*num_fig_grid_row)

    axis_toggle = 'on' #@param ['on', 'off'] {type:'string'}
    tick_lable_size = 5 #@param {type:'slider', min:0, max:12, step:0.5}

    for i in range(num_fig_grid_row):
        for j in range(num_fig_grid_col):  
            if num_fig_grid_row == 1:
                if i*num_fig_grid_col+j-1 < pred_size:
                    ax[j].imshow(prediction[0, :, :, i*num_fig_grid_col+j-1], cmap='gray')
                    # ax[j].set_title('Prediction with layer {} and kernel {}'.format(num_layer, i*num_fig_grid_col+j), fontsize=8)
                    plt.rc('xtick',labelsize=tick_lable_size)
                    plt.rc('ytick',labelsize=tick_lable_size)
                    ax[j].axis(axis_toggle)
                else: 
                    plt.rc('xtick',labelsize=tick_lable_size)
                    plt.rc('ytick',labelsize=tick_lable_size)
                    ax[j].axis(axis_toggle)
                
            else: 
                if i*num_fig_grid_col+j-1 < pred_size: # draw prediction with prevention out of prediction size
                    ax[i, j].imshow(prediction[0, :, :, i*num_fig_grid_col+j-1], cmap='gray')
                    # ax[i, j].set_title('Prediction with layer {} and kernel {}'.format(num_layer, i*num_fig_grid_col+j), fontsize=8)
                    plt.rc('xtick',labelsize=tick_lable_size)
                    plt.rc('ytick',labelsize=tick_lable_size)
                    ax[i, j].axis(axis_toggle)
                else:
                    plt.rc('xtick',labelsize=tick_lable_size)
                    plt.rc('ytick',labelsize=tick_lable_size)
                    ax[i, j].axis(axis_toggle)
    plt.show()  
    
MODEL = 'generator' #@param ['generator', 'discriminator'] {type:'string'} 
num_layer = 24 #@param  {type:'slider', min:0, max:30, step:1} 
num_column = 16 #@param  {type:'slider', min:0, max:64, step:2} 

for example_input, example_target in test_dataset.take(1):
    for i in range(num_layer):
        generate_feature_map(model_type=MODEL, num_layer=i, test_input=example_input, tar=example_target, 
                             num_fig_grid_col=num_column) 

## Restore the latest checkpoint and test

In [ ]:
!ls {checkpoint_dir}
print(checkpoint_dir)

In [ ]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

## Create with the given design conditions
The condition images should be placed in the val dataset; this cell will use the images to generate the corresponding solutions.

In [ ]:
# Run the trained model on the provided example (design condition) from the val dataset
for inp, tar in val_dataset.take(-1): ######## generate predictive image by testing the model with one image (or more) within the training set ########-----------------------
  generate_images(generator, inp, tar) # call generate_images() method to finish the job

In [ ]:
# Run the trained model on the provided example (design condition) from the val dataset
for inp, tar in test_dataset.take(-1): ######## generate predictive image by testing the model with one image (or more) within the training set ########-----------------------
  generate_images(generator, inp, tar) # call generate_images() method to finish the job